In [38]:
import numpy as np
import sys, os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import measure
import cv2
from skimage import morphology
from skimage.morphology import disk, square, dilation, erosion
from skimage import data
from skimage.feature import Cascade
from PupilDetector import GradientIntersect
from FaceAligner import FaceAligner
import imutils

In [39]:
kEyePercentTop = 25
kEyePercentSide = 13
kEyePercentHeight = 30
kEyePercentWidth = 35

In [40]:
def getEyesCenter(face_BGR):
    img_GRAY = cv2.cvtColor(face_BGR, cv2.COLOR_BGR2GRAY)
    
    eye_region_width = face.shape[0] * (kEyePercentWidth/100.0)
    eye_region_height = face.shape[0] * (kEyePercentHeight/100.0)
    eye_region_top = face.shape[1] * (kEyePercentTop/100.0)
        
    leftEyeRegion = face.shape[0]*(kEyePercentSide/100.0), eye_region_top, eye_region_width, eye_region_height
    rightEyeRegion = face.shape[0] - eye_region_width - face.shape[0]*(kEyePercentSide/100.0), eye_region_top,eye_region_width,eye_region_height
    
    x1, y1, w1, h1 = (leftEyeRegion)
    region = img_GRAY[int(y1) :int(y1) + int(h1) , int(x1) :int(x1) + int(w1)]
    leftEyeCenter = gi.locate(region)
    
    x2, y2, w2, h2 = (rightEyeRegion)
    region = img_GRAY[int(y2) :int(y2) + int(h2) , int(x2) :int(x2) + int(w2)]
    rightEyeCenter = gi.locate(region)
    #rightEyeCenter = np.asarray(rightEyeCenter, dtype="int")
    
    return (int(leftEyeCenter[1])+int(x1) , int(leftEyeCenter[0])+int(y1)), (int(rightEyeCenter[1])+int(x2), int(rightEyeCenter[0])+int(y2))

In [41]:
haar_file = 'haarcascade_frontalface_default.xml'
webcam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(haar_file)
cv2.namedWindow("After Alignment", cv2.WINDOW_NORMAL)
cv2.namedWindow("Before Alignment", cv2.WINDOW_NORMAL)
gi = GradientIntersect()
leftEyeCenter = (0, 0)
rightEyeCenter = (0, 0)
leftEyeRegionStart = (0, 0)
rightEyeRegionStart= (0, 0)
face_rect = (0, 0, 0, 0)
fa = FaceAligner(desiredFaceWidth=256)
while True:
    (_, img_BGR) = webcam.read()
    img = img_BGR.copy()
    
    img_GRAY = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img_GRAY, 1.3, 4)
    
    for (x, y, w, h) in faces:
        face_rect = (x, y, w, h)

        face = img_BGR[y :y + h , x :x + w ]
        
        face = cv2.resize(face, (256, 256))
        (leftEyeCenter, rightEyeCenter) = getEyesCenter(face)
        
        #img_BGR = cv2.circle(img_BGR, (leftEyeCenter[0]+x, leftEyeCenter[1]+y), 1, (0,0,255), -1)
        #img_BGR = cv2.circle(img_BGR, (rightEyeCenter[0]+x, rightEyeCenter[1]+y), 1, (0,0,255), -1)
        
        face = cv2.circle(face, (leftEyeCenter[0], leftEyeCenter[1]), 1, (0,0,255), -1)
        face = cv2.circle(face, (rightEyeCenter[0], rightEyeCenter[1]), 1, (0,0,255), -1)
        
        faceAligned = fa.align(face, rightEyeCenter, leftEyeCenter)
        
    cv2.imshow('After Alignment',faceAligned)
    cv2.imshow('Before Alignment',img_BGR)
    key = cv2.waitKey(5)    
    if key == 27:
        break
    
cv2.destroyAllWindows()
webcam.release()